# Membangun dengan Model Keluarga Meta

## Pendahuluan

Pelajaran ini akan membahas:

- Mengenal dua model utama dari keluarga Meta - Llama 3.1 dan Llama 3.2
- Memahami kasus penggunaan dan skenario untuk masing-masing model
- Contoh kode untuk menunjukkan fitur unik dari setiap model

## Keluarga Model Meta

Dalam pelajaran ini, kita akan membahas 2 model dari keluarga Meta atau "Llama Herd" - Llama 3.1 dan Llama 3.2

Model-model ini tersedia dalam beberapa varian dan bisa ditemukan di marketplace Model Github. Berikut informasi lebih lanjut tentang cara menggunakan Github Models untuk [prototyping dengan model AI](https://docs.github.com/en/github-models/prototyping-with-ai-models?WT.mc_id=academic-105485-koreyst).

Varian Model:
- Llama 3.1 - 70B Instruct
- Llama 3.1 - 405B Instruct
- Llama 3.2 - 11B Vision Instruct
- Llama 3.2 - 90B Vision Instruct

*Catatan: Llama 3 juga tersedia di Github Models namun tidak akan dibahas dalam pelajaran ini*


## Llama 3.1

Dengan 405 Miliar Parameter, Llama 3.1 masuk ke dalam kategori LLM open source.

Model ini merupakan peningkatan dari Llama 3 sebelumnya dengan menawarkan:

- Jendela konteks yang lebih besar - 128k token dibandingkan 8k token
- Maksimal Output Token yang lebih besar - 4096 dibandingkan 2048
- Dukungan Multibahasa yang lebih baik - karena peningkatan jumlah token pelatihan

Hal-hal ini memungkinkan Llama 3.1 menangani kasus penggunaan yang lebih kompleks saat membangun aplikasi GenAI, termasuk:
- Native Function Calling - kemampuan untuk memanggil alat dan fungsi eksternal di luar alur kerja LLM
- Performa RAG yang lebih baik - berkat jendela konteks yang lebih besar
- Pembuatan Data Sintetis - kemampuan untuk membuat data yang efektif untuk tugas seperti fine-tuning


### Native Function Calling

Llama 3.1 telah disesuaikan agar lebih efektif dalam melakukan pemanggilan fungsi atau alat. Model ini juga memiliki dua alat bawaan yang bisa dikenali dan digunakan sesuai permintaan dari pengguna. Alat-alat tersebut adalah:

- **Brave Search** - Dapat digunakan untuk mendapatkan informasi terbaru seperti cuaca dengan melakukan pencarian web
- **Wolfram Alpha** - Dapat digunakan untuk perhitungan matematika yang lebih kompleks sehingga Anda tidak perlu menulis fungsi sendiri.

Anda juga bisa membuat alat kustom sendiri yang dapat dipanggil oleh LLM.

Pada contoh kode di bawah ini:

- Kita mendefinisikan alat yang tersedia (brave_search, wolfram_alpha) di prompt sistem.
- Mengirim prompt pengguna yang menanyakan tentang cuaca di sebuah kota tertentu.
- LLM akan merespons dengan memanggil alat Brave Search yang akan terlihat seperti ini `<|python_tag|>brave_search.call(query="Stockholm weather")`

*Catatan: Contoh ini hanya melakukan pemanggilan alat, jika Anda ingin mendapatkan hasilnya, Anda perlu membuat akun gratis di halaman API Brave dan mendefinisikan fungsi tersebut sendiri*


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import AssistantMessage, SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "meta-llama-3.1-405b-instruct"

client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)


tool_prompt=f"""
<|begin_of_text|><|start_header_id|>system<|end_header_id|>

Environment: ipython
Tools: brave_search, wolfram_alpha
Cutting Knowledge Date: December 2023
Today Date: 23 July 2024

You are a helpful assistant<|eot_id|>
"""

messages = [
    SystemMessage(content=tool_prompt),
    UserMessage(content="What is the weather in Stockholm?"),

]

response = client.complete(messages=messages, model=model_name)

print(response.choices[0].message.content)





### Llama 3.2

Meskipun merupakan sebuah LLM, salah satu keterbatasan yang dimiliki Llama 3.1 adalah multimodalitas. Artinya, kemampuan untuk menggunakan berbagai jenis input seperti gambar sebagai prompt dan memberikan respons. Kemampuan ini menjadi salah satu fitur utama dari Llama 3.2. Fitur-fitur tersebut juga mencakup:

- Multimodalitas - memiliki kemampuan untuk mengevaluasi prompt berupa teks maupun gambar
- Varian ukuran Kecil hingga Menengah (11B dan 90B) - memberikan opsi penerapan yang fleksibel,
- Varian khusus teks (1B dan 3B) - memungkinkan model dijalankan di perangkat edge / mobile dan memberikan latensi rendah

Dukungan multimodal ini merupakan langkah besar di dunia model open source. Contoh kode di bawah ini menggunakan gambar dan prompt teks untuk mendapatkan analisis gambar dari Llama 3.2 90B.

### Dukungan Multimodal dengan Llama 3.2


In [ ]:
%pip install azure-core
%pip install azure-ai-inference

In [ ]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import (
    SystemMessage,
    UserMessage,
    TextContentItem,
    ImageContentItem,
    ImageUrl,
    ImageDetailLevel,
)
from azure.core.credentials import AzureKeyCredential

token = os.environ["GITHUB_TOKEN"]
endpoint = "https://models.inference.ai.azure.com"
model_name = "Llama-3.2-90B-Vision-Instruct"


client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

response = client.complete(
    messages=[
        SystemMessage(
            content="You are a helpful assistant that describes images in details."
        ),
        UserMessage(
            content=[
                TextContentItem(text="What's in this image?"),
                ImageContentItem(
                    image_url=ImageUrl.load(
                        image_file="sample.jpg",
                        image_format="jpg",
                        detail=ImageDetailLevel.LOW)
                ),
            ],
        ),
    ],
    model=model_name,
)

print(response.choices[0].message.content)

## Belajar tidak berhenti di sini, lanjutkan Perjalananmu

Setelah menyelesaikan pelajaran ini, lihat [Koleksi Pembelajaran AI Generatif](https://aka.ms/genai-collection?WT.mc_id=academic-105485-koreyst) kami untuk terus meningkatkan pengetahuanmu tentang AI Generatif!



---

**Penafian**:  
Dokumen ini telah diterjemahkan menggunakan layanan terjemahan AI [Co-op Translator](https://github.com/Azure/co-op-translator). Meskipun kami berupaya untuk memberikan terjemahan yang akurat, harap diketahui bahwa terjemahan otomatis dapat mengandung kesalahan atau ketidakakuratan. Dokumen asli dalam bahasa aslinya harus dianggap sebagai sumber yang berwenang. Untuk informasi yang bersifat kritis, disarankan menggunakan jasa terjemahan profesional oleh manusia. Kami tidak bertanggung jawab atas kesalahpahaman atau penafsiran yang timbul akibat penggunaan terjemahan ini.
